# Задание №2: Фрактальное сжатие

ФИО: Трифонов Андрей Рафисович 
Группа: 213

**Баллы за задание складываются из двух частей: баллы за выполнение промежуточных подзаданий и баллы за качество**

**Максимальное количество баллов за выполнение промежуточных подзаданий — 15**

**Баллы за качество выставляются по итогам сравнения всех решений**

## Правила сдачи
* У каждого подзадания указано максимальное количество баллов, которые можно за него получить
* Для сдачи необходимо в Google Classroom загрузить Jupyter-ноутбук с выполненными подзаданиями
* В некоторых ячейках есть строки (`# GRADED CELL: [function name]`), эти строки **менять нельзя**, они будет использоваться при проверке вашего решения
* Интерфейс функций и классов помеченных таким образом должен остаться без изменений
* Ячейка со строкой (`# GRADED CELL: [function name]`) должна содержать только **одну функцию или класс**
 * Лайфхак: функции можно определять внутри функций
* Никакие другие ячейки не будут использованы при проверке, они должны быть самодостаточны
* Запрещено импортировать иные библиотеки и функции, кроме указанных в первой ячейке с кодом  
(если сильно захочется что-то еще импортировать, спросите в чате курса)

## Немного теории
Алгоритм описан в главе про [сжатие изображений](https://compression.ru/book/part2/part2__3.htm#_Toc448152512).

### Определения
**Ранговый блок**: если исходное изображение разбивается на непересекающиеся блоки одинакового размера, замощающие всё изображение, то каждый такой блок называется *ранговым*; имеют меньший размер, чем доменные блоки.

**Доменный блок**: если исходное изображение разбивается блоки одинакового размера, которые могут и пересекаться, то каждый такой блок называется *доменным*; имеют больший размер, чем ранговые блоки.

**Идея алгоритма**:

При сжатии:
1. для каждого рангового блока найти наиболее похожий на него доменный блок (с учётом поворотов и симметрии)
2. выполнить преобразование яркости
3. в качестве сжатого изображения выступают коэффициенты преобразования ранговых блоков, эффективно записанные в файл (строку)

При декомпрессии:
1. Прочитать файл (строку), извлечь коэффициенты преобразований
2. Применить преобразования к исходному изображению (обычно просто серое) пока результат не стабилизируется

In [ ]:
# Standard Python Library

import os
import itertools

from collections import namedtuple

# Additional Modules

import matplotlib.pyplot as plt
import numpy as np

from skimage import io
from skimage import data, img_as_float64
from skimage.metrics import mean_squared_error as mse, peak_signal_noise_ratio as psnr
from skimage.transform import resize
from skimage.color import rgb2gray, rgb2yuv, yuv2rgb

from tqdm import tqdm

Первым делом нужно загрузить картинку

In [ ]:
lenna_rgb_512x512 = io.imread('test_files/lenna.bmp')
lenna_rgb_256x256 = resize(lenna_rgb_512x512, (256, 256))
lenna_gray_256x256 = np.rint(rgb2gray(lenna_rgb_256x256) * 255).astype('uint8')

`plt` — модуль для рисования графиков и всего остального

Очень удобная штука, будем пользоваться ей довольно часто

In [ ]:
plt.imshow(lenna_gray_256x256, cmap='gray')

## Общие функции
В следующих клетках описаны функции и классы, которые будут использоваться **вами** при выполнении следующих подзаданий. Стоит с ними подробно ознакомиться, понять, что они делают, и поэкспериментировать.

**Не следует их менять.**

In [ ]:
BlockTransform = namedtuple('BlockTransform', ['x', 'y', 'di', 'tr'])
FractalCompressionParams = namedtuple(
    'FractalCompressionParams', [
        'height',
        'width',
        'is_colored',
        'block_size',
        'spatial_scale',
        'intensity_scale',
        'stride'
    ]
)

In [ ]:
def derive_num_bits(length, stride):
    return np.ceil(np.log2(length / stride)).astype(int)

In [ ]:
def is_colored(image):
    if len(image.shape) == 2:
        return False
    elif len(image.shape) == 3 and image.shape[-1] == 3:
        return True
    else:
        message = 'Invalid shape of the image: `{}`'
        raise ValueError(message.format(image.shape))

### [4 балла] Функция для нахождения наилучшего преобразования рангового блока

#### Описание

на входе функции подаются:
* исходное Ч/Б изображение (`image`)
* уменьшенное изображение (`resized_image`)
* координаты рангового блока (`x`, `y`)
* размер блока (`block_size`)
* шаг, через сколько пикселей перескакивать при переборе (`stride`)

на выходе функция должна выдавать:
* лучшее преобразование в смысле MSE, объект типа `BlockTransform`

In [ ]:
# GRADED CELL: find_block_transform

def find_block_transform(image, resized_image, x, y, block_size=8, stride=1, intensity=0.75):
    '''Find best transformation for given rank block.

    Parameters
    ----------
    image : np.array
        Source B/W image.

    resized_image: np.array
        Resized source image.

    x, y: int, int
        Coordinates of the rank block.
    
    block_size: int
        Size of rank block.

    stride: int
        Vertical and horizontal stride for domain block search.
    
    intensity: float
        Brightness scaling.

    Returns
    -------
    best_transform: BlockTransform
        Best transformation.
    '''
    trs = np.arange(8)
    domain_xs = np.arange(0, resized_image.shape[0] - block_size + 1, stride)
    domain_ys = np.arange(0, resized_image.shape[1] - block_size + 1, stride)
    
    def img2domain_blocks(img):
        domain_blocks = []
        
        imgs4trs = np.array([img for tr in trs])
        for tr in range(1, 8):
            imgs4trs[tr] = np.rot90(imgs4trs[tr - 1])
        for tr in range(4, 8):
            imgs4trs[tr] = np.fliplr(imgs4trs[tr])
        
        for tr, dm_x, dm_y in itertools.product(trs, domain_xs, domain_ys):
            domain_blocks.append(imgs4trs[tr][dm_x:(dm_x + block_size),
                                              dm_y:(dm_y + block_size)])
        return domain_blocks
    
    # we may want to have a prebuilt array of domain blocks
    if ((x, y) == (0, 0)):
        find_block_transform.db_cache = img2domain_blocks(resized_image)
    
    domain_blocks = find_block_transform.db_cache # fetch Dij from cache
    rank_block = image[x:(x + block_size), y:(y + block_size)]
    pixels_in_block = block_size * block_size
    
    min_distance = 100000000000
    domain_block_ID = 0
    # best_block_ID = 0
    for tr, dm_x, dm_y in itertools.product(trs, domain_xs, domain_ys):
        domain_block = domain_blocks[domain_block_ID]
        domain_block_ID += 1
        di = (np.sum(domain_block) - np.sum(rank_block)) / pixels_in_block
        current_distance = np.sum( np.square(rank_block * intensity +
                                            np.full(rank_block.shape, di) -
                                            domain_block) )
        if (current_distance < min_distance):
            min_distance = current_distance
            best_transform = BlockTransform(dm_x, dm_y,
                                            (int)(di), tr)
            #best_block_ID = domain_block_ID
    
    return best_transform #, domain_blocks[best_block_ID]

In [ ]:
# SAMPLE INPUT CELL: find_block_transform

#image = lenna_gray_256x256
#spatial_scale = 0.5
#new_size = (int(image.shape[0] * spatial_scale), int(image.shape[1] * spatial_scale))
#resized_image = resize(image, new_size, anti_aliasing=True)
#x = 8
#y = 112
#block_size = 8
#stride = 1
#intensity = 0.75
#
#rank_block = image[x:(x + block_size), y:(y + block_size)]
#plt.imshow(rank_block, cmap='gray')

In [ ]:
# TESTING CELL: find_block_transform
#tr, best_block = find_block_transform(image, resized_image, x, y, block_size=8, stride=1, intensity=0.75)
#plt.imshow(best_block, cmap='gray')
#tr

### [4 балла] Применение IFS к изображению

#### Описание

на входе функции подаются:
* исходное изображение (`image`)
* уменьшенное изображение (`resized_image`)
* IFS, массив объектов типа `BlockTransform` (`transforms`)
* размер блока (`block_size`)

на выходе функция должна выдавать:
* картинку после одинарного применения IFS

In [ ]:
# GRADED CELL: perform_transform

def perform_transform(image, resized_image, transforms, block_size):
    '''Perform IFS on given image.
    
    Parameters
    ----------
    image : np.array
        Source image.

    resized_image: np.array
        Resized source image.

    transforms: list of BlockTransform's
        Given IFS, Iterated Function System
    
    block_size: int
        Size of rank block.

    Returns
    -------
    transformed_image: np.array
        Transformed image.
    '''
    
    # genrating and image for all 90˚ rotations + mirror
    def img_4_blocks(img):
        trs = np.arange(8) 
        img_and_trs = np.array([resized_image for tr in trs])
        
        for tr in range(1, 8):
            img_and_trs[tr] = np.rot90(img_and_trs[tr - 1])
        for tr in range(4, 8):
            img_and_trs[tr] = np.fliplr(img_and_trs[tr])
            
        return img_and_trs
    
    xs = np.arange(0, image.shape[0], block_size)
    ys = np.arange(0, image.shape[1], block_size)
    
    transformed_image = image
    
    transform_ID = 0
    imgs = img_4_blocks(resized_image)

    for x, y in itertools.product(xs, ys):
        t = transforms[transform_ID]
        transform_ID += 1
        transformed_image[x:(x + block_size),
                          y:(y + block_size)
            ] = -t.di + 0.75 * imgs[t.tr][t.x:(t.x + block_size), t.y:(t.y + block_size)]

    return transformed_image

In [ ]:
out = comp.decompress(result_16x4, 2)
plt.imshow(out, cmap='gray')

### [7 баллов] Класс, реализующий интерфейс битового массива
Он понадобится для преобразования найденной IFS в строку, чтобы записать сжатый файл на диск.

In [ ]:
# GRADED CELL: BitBuffer

class BitBuffer:
    '''Class that provides storing and and reading integer numbers 
    in continuous bytearray.
    
    !!! BitBuffer is FIFO data structure !!!

    Parameters
    ----------
    buffer : bytearray, optional (default=None)
        Input bytearray, for initialization.

    Attributes
    ----------
    _buffer : bytearray
        Bytearray that can contain any information.

    Examples
    --------
    >>> buffer = BitBuffer()
    >>> buffer.push(1, 1)
    >>> x = buffer.pop(1)
    >>> print(x)
    1
    >>> buffer.push(125, 18)
    >>> x = buffer.pop(18)
    >>> print(x)
    125
    >>> buffer.push(5, 3)
    >>> x = buffer.pop(3)
    >>> print(x)
    5

    >>> dy = transform.y // stride
    >>> buffer.push(dy, self._num_bits_ver)
    '''

    def __init__(self, buffer=None):
        self._buffer = buffer or bytearray()
        # already (written/read) bits by previous (push/pop)
        self._bits2ignore_push = 0
        self._bits2ignore_pop = 0

    def to_bytearray(self):
        '''Convert to bytearray.
    
        Returns
        -------
        buffer: bytearray
            Bytearray that contains all data.
        '''
        
        return self._buffer

    def push(self, x, n_bits):
        '''Push given integer to buffer. (append)
    
        Parameters
        ----------
        x : int
            Input number.

        n_bits: int
            Number of bits for store input number,
            should be greater than log2(x).
        '''
        
        # shift x's bits closer to ending of written bits (b)
        #                         (i i i) <- self._bits2ignore_push (written already)
        # ... | b b b b  b b b b | b b b -  - - - - |
        #                                x  x x 0 0 <- gap_remover (0)
        
        # (gap_remover == 8) <= (self._bits2ignore_push + n_bits == 8 * n), n > 0
        # if so ending will be byte-like
        
        #print("debug dump 4 x=", x)
        gap_remover = 8 - (self._bits2ignore_push + n_bits) % 8 # [0; 8]
        not_byte_sized = 1 if (gap_remover != 8) else 0
        gap_remover %= 8 # preventing 8 bit x shift
        x <<= gap_remover
        
        bytes_to_push = (self._bits2ignore_push + n_bits) // 8 + not_byte_sized
        x = np.int64(x)
        x_bytes = (x.item()).to_bytes(bytes_to_push, byteorder="big", signed=False)
        
        # merge x's first and buffer's last byte
        if (self._bits2ignore_push):
            #print("gap remover:", gap_remover)
            #print("2ignore :", self._bits2ignore_push)
            #print("ending of buffer: ", self._buffer[-1], "; ending of x_bytes: ", x_bytes)
            self._buffer[-1] += x_bytes[0]
            x_bytes = x_bytes[1:] # we dont want to append x's first later
        
        for byte in x_bytes:
            self._buffer.append(byte)
        
        self._bits2ignore_push = (8 - gap_remover) % 8
        
    def pop(self, n_bits):
        '''Pop n_bits from buffer and transform it to a number.
    
        Parameters
        ----------
        n_bits: int
            Number of bits for pop from buffer.

        Returns
        -------
        x: int
            Extracted number.
        '''
        
        #print(self._bits2ignore_pop)
        
        #  (i i i)  <- self._bits2ignore_pop (read already)
        # | ? ? ? x  x x x x | x b b b  b b b b | ...
        # | ? ? ? x  x x x x | x(b b b  b b b b) <- aligner
        # | ? ? ? ?  ? ? ? ? | ? ? x  x x x x x
        
        # (aligner == 8) <= (self._bits2ignore_pop + n_bits == 8 * n), n > 0
        # if so beginning will be byte-like
        
        aligner = 8 - (self._bits2ignore_pop + n_bits) % 8 # [0; 8]
        not_byte_sized = 1 if (aligner != 8) else 0
        aligner %= 8 # preventing 8 bit x shift
        
        bytes_to_pop = (self._bits2ignore_pop + n_bits) // 8
        bytes_to_read = bytes_to_pop + not_byte_sized
        x_bytes = self._buffer[:bytes_to_read]
        
        x = int.from_bytes(x_bytes, byteorder='big', signed=False)
        x >>= aligner
        x &= 2 ** n_bits - 1
        
        self._buffer = self._buffer[bytes_to_pop:]
        self._bits2ignore_pop = (8 - aligner) % 8
        
        return x

### [Баллы за качество] Класс, реализующий интерфейс архиватора изображений

#### Условие
* Класс будет тестироваться как на черно-белых, так и на **цветных** изображениях
* Для цветных изображений необходимо переходить в YUV, сжимать, а потом обратно в RGB для финального результата
* В качестве оценки алгоритма будет использоваться кривая размер-качество, построенная на основе запуска метода compress2, с параметрами качества [0, 20, 40, 60, 80, 100]
* Следует обеспечить непрерывную монотонную зависимость реального качества декодированного изображения от параметра качества
* Баллы будут выставляться исходя из того, насколько построенный график размер-качество лежит близко к верхнему левому углу (высокое качество и низкий размер)
* За красивые графики с равномерно распределенными узлами [0 ... 100] и без точек перегиба выставляются дополнительные баллы
* Ограничение времени работы (суммарно сжатие и разжатие) на всех уровнях качества: 8 минут

**Интерфейсом данного класса считаются только методы compress2 и decompress, остальные можно менять как угодно**`

In [ ]:
# GRADED CELL: FractalCompressor

class FractalCompressor:
    '''Class that performs fractal compression/decompression of images.

    Attributes
    ----------
    _num_bits_ver : int
        Number of bits for store VERTICAL OFFSET for each transformation.
    
    _num_bits_hor : int
        Number of bits for store HORIZONTAL OFFSET for each transformation.
        
    _num_bits_pix : int
        Number of bits for store INTENSITY OFFSET for each transformation.
        
    _num_bits_tfm : int
        Number of bits for store TRANFORMATION INDEX for each transformation.

    Examples
    --------
    >>> comp = FractalCompressor()
    >>> compressed_image = comp.compress(image, block_size=8, stride=2)
    >>> decompressed_image = comp.decompress(compressed_image, num_iters=9)
    >>> yet_another_compressed_image = comp.compress(image, 8, 4, 0.5, 0.7)
    >>> yet_another_decompressed_image = comp.compress(yet_another_compressed_image, 5)
    '''

    def __init__(self):
        self._num_bits_ver = None
        self._num_bits_hor = None
        self._num_bits_pix = 9
        self._num_bits_tfm = 3
    
    # HEADER TAKES 7 BYTES OF BUFFER
    # Binary header example:
    # |00100000|000 00100|000000 0 0|00000100|0 0110010| 1001011 0|00000001| ...
    
    def _add_header(self, buffer, params):
        '''Store header in buffer.
    
        Parameters
        ----------
        buffer: BitBuffer
            
        params: FractalCompressionParams
            Parameters that should be stored in buffer.

        Note
        ----
        This method must be consistent with `_read_header`.
        '''
        
        buffer.push(params.height, 11)
        buffer.push(params.width, 11)
        buffer.push(1 if (params.is_colored) else 0, 1)
        buffer.push(params.block_size, 10)
        buffer.push(int(params.spatial_scale * 100), 7)
        buffer.push(int(params.intensity_scale * 100), 7)
        buffer.push(params.stride, 9)
    
    def _read_header(self, buffer):
        '''Read header from buffer.
    
        Parameters
        ----------
        buffer: BitiBuffer

        Returns
        -------
        params: FractalCompressionParams
            Extracted parameters.
            
        Note
        ----
        This method must be consistent with `_add_header`.
        '''
        
        params = FractalCompressionParams(
            buffer.pop(11),
            buffer.pop(11),
            buffer.pop(1) == 1,
            buffer.pop(10),
            buffer.pop(7) / 100,
            buffer.pop(7) / 100,
            buffer.pop(9)
            )
            
        return params
    
    def _add_to_buffer(self, buffer, transform, stride):
        '''Store block transformation in buffer.
    
        Parameters
        ----------
        buffer: BitBuffer

        transform: BlockTransform
            
        stride: int
            Vertical and horizontal stride for domain block search.

        Note
        ----
        This method must be consistent with `_read_transform`.
        '''
        
        di_negative_flag = 1 if (transform.di < 0) else 0
        di_unsigned = (abs(transform.di) << 1) + di_negative_flag
        
        buffer.push((int)( (transform.x / stride)  ), self._num_bits_hor)
        buffer.push((int)( (transform.y / stride)  ), self._num_bits_ver)
        buffer.push((int)(  di_unsigned            ), self._num_bits_pix)
        buffer.push((int)( (transform.tr)          ), self._num_bits_tfm)
        
    def _read_transform(self, buffer, stride):
        '''Read block transformation from buffer.
    
        Parameters
        ----------
        buffer: BitBuffer

            
        stride: int
            Vertical and horizontal stride for domain block search.
            
        Returns
        -------
        transform: BlockTransform
            Extracted block transformation.

        Note
        ----
        This method must be consistent with `_add_to_buffer`.
        '''

        x = buffer.pop(self._num_bits_hor) * stride
        y = buffer.pop(self._num_bits_ver) * stride
        di_unsigned = buffer.pop(self._num_bits_pix)
        di_sign = -1 if (di_unsigned % 2 == 1) else 1
        
        transform = BlockTransform (
            x, y,
            di_sign * (di_unsigned >> 1),
            buffer.pop(self._num_bits_tfm)
        )
    
        return transform
    
    def _ifs2buf(self, params, transformations):
        '''Store compression parameters and IFS in buffer.
    
        Parameters
        ----------
        params: FractalCompressionParams
            Parameters of the compression.

        transformations: list of BlockTransform's
            Given IFS.

        Returns
        -------
        buffer: BitBuffer

        Note
        ----
        This method must be consistent with `_buf2ifs`.
        '''
        
        buffer = BitBuffer()
        self._add_header(buffer, params)
        
        self._num_bits_ver = derive_num_bits(params.height, params.stride)
        self._num_bits_hor = derive_num_bits(params.width, params.stride)
        
        for t in transformations:
            self._add_to_buffer(buffer, t, params.stride)
        
        return buffer
    
    def _buf2ifs(self, buffer):
        '''Store compression parameters and IFS in buffer.
    
        Parameters
        ----------
        buffer: BitBuffer

        Returns
        -------
        params: FractalCompressionParams
            Extracted compression parameters.

        transforms: list of BlockTransform's
            Extracted IFS.

        Note
        ----
        This method must be consistent with `_ifs2buf`.
        '''

        params = self._read_header(buffer)
        
        self._num_bits_ver = derive_num_bits(params.height, params.stride)
        self._num_bits_hor = derive_num_bits(params.width, params.stride)
        
        num_transforms = int(params.height * params.width / params.block_size ** 2)
        transforms = [
            self._read_transform(buffer, params.stride)
            for _ in range(num_transforms)
        ]
        
        return params, transforms
        
    def compress(self, image, block_size=8, stride=1,
                 spatial_scale=0.5, intensity_scale=0.75):
        '''Compress input image
        
        Parameters
        ----------
        image : np.array
            Source image.

        block_size: int, optional (default = 8)
            Size of rank block.

        stride: int, optional (default = 1)
            Vertical and horizontal stride for domain block search.
        
        spatial_scale : float, optional (default = 0.5)
            ({rank block size} / {domain block size}) ratio, must be < 1.
        
        intensity_scale : float, optional (default = 0.75)
            Reduce coefficient for image intensity.

        Returns
        -------
        byte_array: bytearray
            Compressed image.
            
        Note
        ----
        This method must be consistent with `decompress`.
        '''
        spatial_scale = round(spatial_scale, 2)
        intensity_scale = round(intensity_scale, 2)
        
        new_size = (
            int(image.shape[0] * spatial_scale),
            int(image.shape[1] * spatial_scale)
        )
        resized_image = resize(image, new_size, anti_aliasing=False, preserve_range=True)
        
        xs = np.arange(0, image.shape[0], block_size)
        ys = np.arange(0, image.shape[1], block_size)
        
        transformations = []
        for x, y in tqdm(itertools.product(xs, ys), total=len(xs)*len(ys)):
            transform = find_block_transform(
                image, resized_image, x, y,
                block_size, stride, intensity_scale
            )
            transformations.append(transform)
        
        params = FractalCompressionParams(
            image.shape[0],
            image.shape[1],
            is_colored(image),
            block_size,
            spatial_scale,
            intensity_scale,
            stride
        )
        
        buffer = self._ifs2buf(params, transformations)
        return buffer.to_bytearray()
            
    def compress2(self, image, quality=50):
        '''Compress input image
        
        Parameters
        ----------
        image : np.array
            Source image.

        quality: int, optional (default=50)
            Quality of image compression

        Returns
        -------
        byte_array: bytearray
            Compressed image.
            
        Note
        ----
        This method must be consistent with `decompress`.
        '''
            
        # FEEL FREE TO CHANGE CODE BELOW
            
        return self.compress(image)
    
    def decompress(self, byte_array, num_iters=10):
        '''Decompress input image
        
        Parameters
        ----------
        byte_array: bytearray
            Compressed image.

        num_iters: int, optional (default=10)
            Number of iterations to perform IFS.

        Returns
        -------
        image: np.array
            Decompressed image.
            
        Note
        ----
        This method must be consistent with `compress`.
        '''
        
        params, transforms = self._buf2ifs(BitBuffer(byte_array))
        
        # inital image is black
        image = lenna_gray_256x256 - lenna_gray_256x256
        
        new_size = (
            int(image.shape[0] * params.spatial_scale),
            int(image.shape[1] * params.spatial_scale)
        )
        
        for i in range(num_iters):
            resized_image = resize(image, new_size, anti_aliasing=False,preserve_range=True)
            image = perform_transform(
                    image, resized_image,
                    transforms, params.block_size
            )
        
        return image

## Пробуем применить FractalCompressor

In [ ]:
comp = FractalCompressor()

In [ ]:
result_16x4 = comp.compress(lenna_gray_256x256, block_size=16, stride=4)

Размер сжатого изображения в байтах == длина полученного массива `bytearray`

In [ ]:
len(result_16x4)

### Эволюция изображения при декомпрессии
Выглядит как увеличение фотографии в CSI: Место прреступления

In [ ]:
plt.imshow(lenna_gray_256x256, cmap='gray')

In [ ]:
n_iters = [1, 2, 4, 8]

imgs = [comp.decompress(result_16x4, n) for n in n_iters]
_, axs = plt.subplots(ncols=len(imgs), figsize=(18, 6))
for index in range(len(imgs)):
    axs[index].imshow(imgs[index], cmap='gray')

plt.show()

### Поиграемся с параметрами сжатия
Понятно, что при увеличении перебора мы, во-первых, увеличиваем время вычислений, а во-вторых, улучшаем итоговое качество изображения после сжатия и декомпрессии.

Чтобы увеличить перебор можно уменьшить размер шага `stride` или уменьшить размер доменного блока `block_size`. Но не рекомендуется делать блок размером меньше 4х4.

In [ ]:
result_16x2 = comp.compress(lenna_gray_256x256, block_size=16, stride=2)

In [ ]:
n_iters = [1, 2, 4, 8]

imgs = [comp.decompress(result_16x2, n) for n in n_iters]
_, axs = plt.subplots(ncols=len(imgs), figsize=(18, 6))
for index in range(len(imgs)):
    axs[index].imshow(imgs[index], cmap='gray')

plt.show()

In [ ]:
result_8x4 = comp.compress(lenna_gray_256x256, block_size=8, stride=4)

In [ ]:
n_iters = [1, 2, 4, 8]

imgs = [comp.decompress(result_8x4, n) for n in n_iters]
_, axs = plt.subplots(ncols=len(imgs), figsize=(18, 6))
for index in range(len(imgs)):
    axs[index].imshow(imgs[index], cmap='gray')

plt.show()

In [ ]:
result_8x2 = comp.compress(lenna_gray_256x256, block_size=8, stride=2)

In [ ]:
n_iters = [1, 2, 4, 8]

imgs = [comp.decompress(result_8x2, n) for n in n_iters]
_, axs = plt.subplots(ncols=len(imgs), figsize=(18, 6))
for index in range(len(imgs)):
    axs[index].imshow(imgs[index], cmap='gray')

plt.show()

## Построим график качества
Качество в данном случае будет измеряться по PSNR (а значит в децибелах).

Это базовый график для понимания соотношения между коэффициентом сжатия и качеством, получаемым на выходе. Можно посмотреть, как он будет меняться в зависимости от количества итераций при декомпрессии, например.

In [ ]:
def weighted_psnr(ref, img):
    assert ref.shape == img.shape, "Shape mismatch"
    if is_colored(img):
        ref_yuv = rgb2yuv(ref)
        img_yuv = rgb2yuv(img)
        
        return (4 * psnr(ref_yuv[..., 0], img_yuv[..., 0]) +
                    psnr(ref_yuv[..., 1], img_yuv[..., 1]) +
                    psnr(ref_yuv[..., 2], img_yuv[..., 2])
               ) / 6
    else:
        return psnr(ref, img)

In [ ]:
quality = [0, 20, 40, 60, 80, 100]

def test_image(img):
    compressed_images = [comp.compress2(img, quality=q) for q in quality]
    decompressed_images = [comp.decompress(compressed) for compressed in compressed_images]
    compression_rates = np.array([len(compressed) for compressed in compressed_images]) / img.size
    psnrs = [weighted_psnr(img, decompressed) for decompressed in decompressed_images]
    return compression_rates, psnrs

In [ ]:
def test_and_plot_collection(collection):
    _, ax = plt.subplots(figsize=(8, 6))

    for image in collection:
        compression_rates, psnrs = test_image(image)
        ax.plot(compression_rates, psnrs, marker='o', ms=10, ls='-.')

    ax.set_xlabel('Compression Rate', fontsize=16)
    ax.set_ylabel('PSNR, dB', fontsize=16)

    plt.show()

In [ ]:
test_and_plot_collection([lenna_gray_256x256])

In [ ]:
collection = []
for image_name in os.listdir('test_files'):
    image = resize(io.imread(os.path.join('test_files', image_name)), (256, 256))
    if is_colored(image):
        image = np.rint(rgb2gray(image) * 255).astype('uint8')
    collection.append(image)
test_and_plot_collection(collection)

## Улучшим алгоритм
Одним из основных способов улучшения сжатия изображений является разбиение картинки не на равные блоки, а на блоки разных размеров. Как дополнительную часть задания, мы предлагаем реализовать разбиение квадродеревом, это позволит более гибко настраивать параметры сжатия и получить лучшие результаты.

<center>Пример разбиения изображения на блоки с использованием квадродерева</center>

Исходное изображение | Разбиение квадродеревом
- | -
![Source image](images/house.jpg) | ![Segmentation](images/quadtree.jpg)